In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek
from scipy.stats import zscore
import joblib

# 데이터 불러오기
data = pd.read_csv("../data/raw/ai4i2020.csv")

print("="*50)
print("원본 데이터 클래스 분포:")
print(data['Machine failure'].value_counts())
print(f"고장률: {data['Machine failure'].mean():.2%}")
print("="*50)

# 1. 이상치 처리 (Z-score 기준)
numeric_cols = ["Air temperature [K]", "Process temperature [K]", 
                "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]

# Z-score 3 이상 값 제거 (고장 데이터는 보존 고려)
normal_data = data[data['Machine failure'] == 0]
fault_data = data[data['Machine failure'] == 1]

# 정상 데이터에서만 이상치 제거
normal_data_cleaned = normal_data[(zscore(normal_data[numeric_cols]).abs() < 3).all(axis=1)]
data = pd.concat([normal_data_cleaned, fault_data], axis=0).reset_index(drop=True)

print(f"이상치 제거 후 데이터 크기: {data.shape}")

# 2. 특징 공학 (Feature Engineering)
# 온도 차이
data['Temp_diff'] = data['Process temperature [K]'] - data['Air temperature [K]']

# 토크와 회전속도의 비율 (파워 관련)
data['Power_factor'] = data['Torque [Nm]'] * data['Rotational speed [rpm]'] / 1000

# Tool wear 구간화
data['Tool_wear_category'] = pd.cut(data['Tool wear [min]'], 
                                     bins=[0, 50, 150, 250], 
                                     labels=['Low', 'Medium', 'High'])

# 3. 범주형 변수 원-핫 인코딩
categorical_cols = ["Type", "Tool_wear_category"]
data = pd.get_dummies(data, columns=categorical_cols, drop_first=False)

# 4. 타겟 / 피처 분리
feature_cols = [col for col in data.columns if col not in 
                ["UDI", "Product ID", "Machine failure", "TWF", "HDF", "PWF", "OSF", "RNF"]]
X = data[feature_cols]
y = data["Machine failure"]

# 5. 먼저 Train/Test 분리 (stratify 유지)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6. Train을 다시 Train/Val로 분리
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

print(f"\n분할 후 클래스 분포:")
print(f"Train - 정상: {(y_train==0).sum()}, 고장: {(y_train==1).sum()}")
print(f"Val - 정상: {(y_val==0).sum()}, 고장: {(y_val==1).sum()}")
print(f"Test - 정상: {(y_test==0).sum()}, 고장: {(y_test==1).sum()}")

# 7. 수치형 컬럼 정규화 (증강 전에 수행)
numeric_cols_extended = numeric_cols + ['Temp_diff', 'Power_factor']
scaler = StandardScaler()

# Train 데이터로 fit하고 모든 데이터에 transform
X_train[numeric_cols_extended] = scaler.fit_transform(X_train[numeric_cols_extended])
X_val[numeric_cols_extended] = scaler.transform(X_val[numeric_cols_extended])
X_test[numeric_cols_extended] = scaler.transform(X_test[numeric_cols_extended])

# 8. SMOTE를 사용한 오버샘플링 (Train 데이터에만 적용)
print("\n"+"="*50)
print("SMOTE 적용 중...")

# 여러 샘플링 전략 시도
sampling_strategies = {
    'auto': SMOTE(random_state=42),  # 1:1 비율
    'not_majority': SMOTE(sampling_strategy='not majority', random_state=42),
    '0.3': SMOTE(sampling_strategy=0.3, random_state=42),  # 고장:정상 = 3:10
    '0.5': SMOTE(sampling_strategy=0.5, random_state=42),  # 고장:정상 = 1:2
}

# 최적의 전략 선택 (필요에 따라 변경)
selected_strategy = '0.3'
smote = sampling_strategies[selected_strategy]

try:
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    print(f"선택된 전략: {selected_strategy}")
    print(f"SMOTE 후 Train - 정상: {(y_train_resampled==0).sum()}, 고장: {(y_train_resampled==1).sum()}")
    print(f"새로운 고장률: {y_train_resampled.mean():.2%}")
    
    # SMOTE 적용된 데이터로 교체
    X_train = X_train_resampled
    y_train = y_train_resampled
    
except Exception as e:
    print(f"SMOTE 적용 실패: {e}")
    print("원본 데이터 유지")

# 9. 데이터 저장
print("\n"+"="*50)
print("데이터 저장 중...")

# 데이터 저장
X_train.to_csv("../data/X_train_improved.csv", index=False)
X_val.to_csv("../data/X_val_improved.csv", index=False)
X_test.to_csv("../data/X_test_improved.csv", index=False)
y_train.to_csv("../data/y_train_improved.csv", index=False)
y_val.to_csv("../data/y_val_improved.csv", index=False)
y_test.to_csv("../data/y_test_improved.csv", index=False)

# Scaler 저장 (나중에 예측시 사용)
joblib.dump(scaler, '../models/scaler.pkl')
joblib.dump(feature_cols, '../models/feature_cols.pkl')

print("데이터 전처리 완료!")
print(f"최종 Train 크기: {X_train.shape}")
print(f"최종 Val 크기: {X_val.shape}")
print(f"최종 Test 크기: {X_test.shape}")

원본 데이터 클래스 분포:
Machine failure
0    9661
1     339
Name: count, dtype: int64
고장률: 3.39%
이상치 제거 후 데이터 크기: (9845, 14)

분할 후 클래스 분포:
Train - 정상: 6083, 고장: 217
Val - 정상: 1522, 고장: 54
Test - 정상: 1901, 고장: 68

SMOTE 적용 중...
선택된 전략: 0.3
SMOTE 후 Train - 정상: 6083, 고장: 1824
새로운 고장률: 23.07%

데이터 저장 중...
데이터 전처리 완료!
최종 Train 크기: (7907, 13)
최종 Val 크기: (1576, 13)
최종 Test 크기: (1969, 13)


/opt/anaconda3/envs/smg/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
